In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import InputLayer
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
df_Berk = pd.read_csv('Avg.csv')
df_Berk.head()

,time,latitude,longitude,temperature
0,1750-01-01,26.5,-122.5,NaN
1,1750-01-01,26.5,-121.5,NaN
2,1750-01-01,26.5,-120.5,NaN
3,1750-01-01,26.5,-119.5,NaN
4,1750-01-01,26.5,-118.5,NaN


In [3]:
df_Berk['time'] = pd.to_datetime(df_Berk['time'])

In [4]:
df_copy = df_Berk[['time', 'temperature']]
df_copy = df_copy.set_index('time')

In [5]:
df_copy = df_copy.loc[df_copy.index > '1900-01-01', :]

In [6]:
df_copy = df_copy.resample('M').mean()

In [7]:
df_copy.describe()

,temperature
count,1471.000000
mean,0.143264
std,1.148851
min,-4.264061
25%,-0.564256
50%,0.101511
75%,0.851870
max,5.005915


In [8]:
df_copy.shape

(1471, 1)

In [9]:
df_copy.head()

,temperature
time,
1900-02-28,-1.760479
1900-03-31,-0.235467
1900-04-30,-0.076263
1900-05-31,0.488469
1900-06-30,0.592415


In [10]:
df_copy.set_index('time').to_numpy()

KeyError: "None of ['time'] are in the columns"

In [ ]:
df_copy.to_numpy()

In [11]:
df_copy = df_copy.reset_index()

In [12]:
def create_dataset(dataset, n_features):
    dataset = dataset.set_index('time')
    dataset = dataset.to_numpy()
    X, y = [], []
    for i in range(len(dataset)):
        end_ix = i + n_features
        if end_ix > len(dataset)-1:
            break
        seq_x, seq_y = dataset[i:end_ix], dataset[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [13]:
X, y = create_dataset(df_copy, 1)

In [14]:
X.shape

(1470, 1, 1)

In [15]:
X

array([[[-1.76047867]],

       [[-0.23546683]],

       [[-0.07626311]],

       ...,

       [[ 1.19876959]],

       [[ 1.33462722]],

       [[ 1.51820082]]])

In [16]:
y.shape

(1470, 1)

In [17]:
y

array([[-0.23546683],
       [-0.07626311],
       [ 0.4884685 ],
       ...,
       [ 1.33462722],
       [ 1.51820082],
       [ 1.34283786]])

In [18]:
X = X.reshape((X.shape[0], X.shape[1], 1))

In [19]:
(X.shape[0], X.shape[1])

(1470, 1)

In [20]:
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(1, 1)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=300, verbose=1)

Epoch 1/300
46/46 [==============================] - 2s 2ms/step - loss: 1.3280
Epoch 2/300
46/46 [==============================] - 0s 2ms/step - loss: 1.3010
Epoch 3/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2690
Epoch 4/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2580
Epoch 5/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2565
Epoch 6/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2555
Epoch 7/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2548
Epoch 8/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2542
Epoch 9/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2551
Epoch 10/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2534
Epoch 11/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2539
Epoch 12/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2523
Epoch 13/300
46/46 [=================

46/46 [==============================] - 0s 2ms/step - loss: 1.2403
Epoch 203/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2399
Epoch 204/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2395
Epoch 205/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2403
Epoch 206/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2398
Epoch 207/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2413
Epoch 208/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2423
Epoch 209/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2409
Epoch 210/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2392
Epoch 211/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2403
Epoch 212/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2392
Epoch 213/300
46/46 [==============================] - 0s 2ms/step - loss: 1.2401
Epoch 214/300
46/46 [=========

In [218]:
train, test = df_copy.iloc[0:train_size,:], df_copy.iloc[train_size:len(df_copy),:]

In [219]:
train.head()

,temperature
time,
1900-02-28,-1.760479
1900-03-31,-0.235467
1900-04-30,-0.076263
1900-05-31,0.488469
1900-06-30,0.592415


In [21]:
def split_sequence(sequence, n_steps):
    sequence = sequence.set_index('time').to_numpy()
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [22]:
X, y = split_sequence(df_copy, 1)

In [23]:
print(X.shape), print(y.shape)

(1470, 1, 1)
(1470, 1)


(None, None)

In [27]:
def df_to_X_y(df, window_size=1):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np) - window_size):
        row = [a for a in df_as_np[i:i + window_size]]
        X.append(row)
        label = df_as_np[i + window_size]
        y.append(label)
    return np.array(X), np.array(y)

In [28]:
X, y = df_to_X_y(df_copy)

In [29]:
X.shape

(1470, 1, 2)

In [30]:
X

array([[[Timestamp('1900-02-28 00:00:00'), -1.760478674977152]],

       [[Timestamp('1900-03-31 00:00:00'), -0.2354668338743358]],

       [[Timestamp('1900-04-30 00:00:00'), -0.0762631149176408]],

       ...,

       [[Timestamp('2022-05-31 00:00:00'), 1.1987695925245485]],

       [[Timestamp('2022-06-30 00:00:00'), 1.3346272189439956]],

       [[Timestamp('2022-07-31 00:00:00'), 1.5182008226089265]]],
      dtype=object)

In [31]:
y.shape

(1470, 2)

In [32]:
y

array([[Timestamp('1900-03-31 00:00:00'), -0.2354668338743358],
       [Timestamp('1900-04-30 00:00:00'), -0.0762631149176408],
       [Timestamp('1900-05-31 00:00:00'), 0.4884685007674814],
       ...,
       [Timestamp('2022-06-30 00:00:00'), 1.3346272189439956],
       [Timestamp('2022-07-31 00:00:00'), 1.5182008226089265],
       [Timestamp('2022-08-31 00:00:00'), 1.342837857654623]],
      dtype=object)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [34]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1102, 1, 2)
(368, 1, 2)
(1102, 2)
(368, 2)


In [38]:
model = Sequential()
model.add(InputLayer((5, 1)))
model.add(LSTM(128))
model.add(Dense(16, 'relu'))
model.add(Dense(8, 'relu'))
model.add(Dense(1, 'linear'))
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=300, verbose=1)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type Timestamp).

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128)               66560     
                                                                 
 dense_1 (Dense)             (None, 16)                2064      
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 68,769
Trainable params: 68,769
Non-trainable params: 0
_________________________________________________________________


In [263]:
trainPredict = model.predict(X_train)
testPredict = model.predict(X_test)

12/12 [==============================] - 0s 3ms/step


In [264]:
trainPredict

array([[-0.00142721],
       [ 0.00112843],
       [-0.00121286],
       ...,
       [ 0.01379752],
       [ 0.07214997],
       [ 0.01100146]], dtype=float32)

In [252]:
y_train

array([[ 1.93630142],
       [ 0.12331945],
       [-0.00242965],
       ...,
       [-0.02535854],
       [ 1.63259508],
       [-0.56602626]])

In [253]:
trainScore = np.sqrt(mean_squared_error(y_train, trainPredict))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(y_test, testPredict))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 1.15 RMSE
Test Score: 1.17 RMSE


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler

np.random.seed(73)

# Apply MinMaxScaler to training and test data. Fit to training data, use to transform train + test

scaler = MinMaxScaler(feature_range=(0,1))
scaled_train_values = scaler.fit_transform(train_series.values.reshape(-1,1))
train_series_scaled = pd.Series(data=scaled_train_values.reshape(1,-1)[0], index=train_series.index)

# Create TimeseriesGenerator using training data and selected hyperparameters

n_input = 24
n_features= 1
sampling_rate = 1
stride = 1
batch_size = 1

train_generator = TimeseriesGenerator(scaled_train_values, 
                                      scaled_train_values, 
                                      length=n_input,
                                      sampling_rate=sampling_rate,
                                      stride=stride,
                                      batch_size=batch_size)

# Build and fit model

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=False, input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(train_generator, epochs=80)

# Generate predictions

lstm_preds = []

batch = scaled_train_values[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

for i in range(len(test_series)):
    pred = model.predict(current_batch)[0]
    lstm_preds.append(pred)
    current_batch = np.append(current_batch[:, 1:, :], [[pred]], axis=1)
    
# Evaluate performance
    
lstm_preds = scaler.inverse_transform(lstm_preds)
lstm_preds = [item for sublist in lstm_preds for item in sublist]
    
test_score = mean_squared_error(test_series.values, lstm_preds)
print(test_score)

In [1]:
test_result = adfuller(df_copy['temperature'])

NameError: name 'adfuller' is not defined